# R2Gen项目学习笔记 - 第二部分：数据处理与文本预处理

## 上节回顾 📚

在第一部分中，我们学习了：
- R2Gen项目的整体架构
- 主函数的执行流程  
- 重要参数的含义
- 深度学习的基础概念

## 本节学习目标 🎯

今天我们将深入学习数据处理的核心模块：

1. **文本分词器 (Tokenizer)** - 如何把医学报告转换成数字
2. **数据加载器 (DataLoader)** - 如何高效管理训练数据
3. **图像预处理** - 如何准备医学影像数据
4. **实际代码演示** - 动手体验数据处理过程

## 为什么数据处理如此重要？ 🤔

就像医生需要先整理病历资料才能诊断一样，AI模型也需要把原始的图像和文字整理成标准格式才能学习！

## 什么是Tokenizer（分词器）？ ✂️

**分词器**是深度学习中处理文本的核心工具，它的作用是：

### 🔄 主要功能
1. **文本清洗** - 去除无用的标点符号和格式
2. **分词切割** - 把句子切分成单词
3. **建立词典** - 为每个单词分配唯一的数字ID
4. **数字转换** - 把文字转换成数字序列

### 🏥 医学报告的特殊性
医学报告有其特殊性：
- 包含专业术语（如"心影增大"、"肺纹理增粗"）
- 格式相对固定（如"心脏：正常大小"）
- 需要标准化处理

### 🎯 为什么需要转换成数字？
计算机只能理解数字，不能直接理解文字。就像：
- "心脏" → 数字 ID: 156
- "正常" → 数字 ID: 89  
- "大小" → 数字 ID: 203

In [1]:
# 让我们用简单的例子理解Tokenizer的工作原理

def simple_tokenizer_demo():
    """
    演示分词器的基本工作流程
    这是一个简化版本，帮助理解核心概念
    """
    
    print("=== 🔧 分词器工作流程演示 ===\n")
    
    # 步骤1: 原始医学报告
    print("📋 步骤1: 原始医学报告")
    raw_report = "心脏大小正常。肺部清晰，无异常发现。"
    print(f"原始报告: '{raw_report}'\n")
    
    # 步骤2: 文本清洗
    print("🧹 步骤2: 文本清洗")
    # 去除标点符号，转小写，分句
    cleaned = raw_report.replace("。", " . ").replace("，", "").lower()
    print(f"清洗后: '{cleaned}'\n")
    
    # 步骤3: 分词
    print("✂️ 步骤3: 分词切割")
    tokens = cleaned.split()
    print(f"分词结果: {tokens}\n")
    
    # 步骤4: 建立词典（简化版）
    print("📖 步骤4: 建立词典")
    # 实际中会统计所有训练数据中的词汇
    vocab = {
        '<unk>': 0,  # 未知词
        '心脏': 1,
        '大小': 2, 
        '正常': 3,
        '肺部': 4,
        '清晰': 5,
        '无': 6,
        '异常': 7,
        '发现': 8,
        '.': 9
    }
    print("词典示例:")
    for word, idx in vocab.items():
        print(f"  '{word}' → {idx}")
    print()
    
    # 步骤5: 转换为数字序列
    print("🔢 步骤5: 转换为数字序列")
    token_ids = []
    for token in tokens:
        if token in vocab:
            token_ids.append(vocab[token])
        else:
            token_ids.append(vocab['<unk>'])  # 未知词用0表示
    
    print(f"最终数字序列: {token_ids}")
    print("对应关系:")
    for i, (token, token_id) in enumerate(zip(tokens, token_ids)):
        print(f"  '{token}' → {token_id}")
    
    print("\n💡 这样计算机就能'理解'医学报告了！")

# 运行演示
simple_tokenizer_demo()

=== 🔧 分词器工作流程演示 ===

📋 步骤1: 原始医学报告
原始报告: '心脏大小正常。肺部清晰，无异常发现。'

🧹 步骤2: 文本清洗
清洗后: '心脏大小正常 . 肺部清晰无异常发现 . '

✂️ 步骤3: 分词切割
分词结果: ['心脏大小正常', '.', '肺部清晰无异常发现', '.']

📖 步骤4: 建立词典
词典示例:
  '<unk>' → 0
  '心脏' → 1
  '大小' → 2
  '正常' → 3
  '肺部' → 4
  '清晰' → 5
  '无' → 6
  '异常' → 7
  '发现' → 8
  '.' → 9

🔢 步骤5: 转换为数字序列
最终数字序列: [0, 9, 0, 9]
对应关系:
  '心脏大小正常' → 0
  '.' → 9
  '肺部清晰无异常发现' → 0
  '.' → 9

💡 这样计算机就能'理解'医学报告了！


## 数据加载器 (DataLoader) 解析 📦

**数据加载器**是训练过程中的"后勤部门"，负责：
- 🔄 批量加载数据
- 🖼️ 图像预处理  
- 📝 文本预处理
- 🔀 数据打乱和采样

In [2]:
# DataLoader 工作原理详解

def explain_dataloader():
    """
    解释数据加载器的工作原理和重要性
    用生活化的比喻帮助理解
    """
    
    print("=== 🏭 DataLoader: 训练数据的'生产流水线' ===\n")
    
    print("🎯 核心作用:")
    print("就像工厂的流水线一样，DataLoader负责:")
    print("• 从仓库（数据集）中取出原材料（原始数据）")
    print("• 按照标准流程加工（预处理）")
    print("• 打包成标准规格（批量处理）")
    print("• 源源不断供应给生产线（模型训练）\n")
    
    print("📊 批量处理 (Batch Processing):")
    print("• 不是一张一张图片处理，而是一次处理16张（batch_size=16）")
    print("• 就像医生不是一个一个看病人，而是安排一批病人同时检查")
    print("• 大大提高了处理效率\n")
    
    print("🖼️ 图像预处理流程:")
    print("训练时:")
    print("  1. Resize(256) - 调整图片大小到256x256")
    print("  2. RandomCrop(224) - 随机裁剪到224x224（数据增强）")
    print("  3. RandomHorizontalFlip() - 随机水平翻转（数据增强）") 
    print("  4. ToTensor() - 转换为张量格式")
    print("  5. Normalize() - 标准化像素值")
    print()
    print("测试时:")
    print("  1. Resize(224, 224) - 直接调整到224x224（不随机）")
    print("  2. ToTensor() - 转换为张量")
    print("  3. Normalize() - 标准化\n")
    
    print("🧮 为什么要标准化？")
    print("• 原始像素值范围: 0-255")  
    print("• 标准化后范围: 大约-2到2之间")
    print("• 这样神经网络训练更稳定，收敛更快")
    print("• 就像把不同单位的数据（米、厘米、毫米）统一成一个标准\n")
    
    print("🔀 数据打乱 (Shuffle):")
    print("• 训练时: shuffle=True（打乱数据顺序）")
    print("• 测试时: shuffle=False（保持固定顺序）")
    print("• 为什么要打乱？避免模型记住数据顺序而不是真正学习规律")

# 运行解释
explain_dataloader()

=== 🏭 DataLoader: 训练数据的'生产流水线' ===

🎯 核心作用:
就像工厂的流水线一样，DataLoader负责:
• 从仓库（数据集）中取出原材料（原始数据）
• 按照标准流程加工（预处理）
• 打包成标准规格（批量处理）
• 源源不断供应给生产线（模型训练）

📊 批量处理 (Batch Processing):
• 不是一张一张图片处理，而是一次处理16张（batch_size=16）
• 就像医生不是一个一个看病人，而是安排一批病人同时检查
• 大大提高了处理效率

🖼️ 图像预处理流程:
训练时:
  1. Resize(256) - 调整图片大小到256x256
  2. RandomCrop(224) - 随机裁剪到224x224（数据增强）
  3. RandomHorizontalFlip() - 随机水平翻转（数据增强）
  4. ToTensor() - 转换为张量格式
  5. Normalize() - 标准化像素值

测试时:
  1. Resize(224, 224) - 直接调整到224x224（不随机）
  2. ToTensor() - 转换为张量
  3. Normalize() - 标准化

🧮 为什么要标准化？
• 原始像素值范围: 0-255
• 标准化后范围: 大约-2到2之间
• 这样神经网络训练更稳定，收敛更快
• 就像把不同单位的数据（米、厘米、毫米）统一成一个标准

🔀 数据打乱 (Shuffle):
• 训练时: shuffle=True（打乱数据顺序）
• 测试时: shuffle=False（保持固定顺序）
• 为什么要打乱？避免模型记住数据顺序而不是真正学习规律


## 实际数据处理演示 🛠️

让我们动手体验一下数据处理的完整流程：

In [3]:
# 模拟完整的数据处理流程

def data_processing_simulation():
    """
    模拟R2Gen中的完整数据处理流程
    让大家体验从原始数据到训练数据的转换过程
    """
    
    print("=== 🔄 完整数据处理流程模拟 ===\n")
    
    # 模拟原始数据
    print("📁 步骤1: 原始数据")
    raw_data = {
        'image_path': 'datasets/iu_xray/images/CXR1_1_IM-0001/1.png',
        'report': '心脏大小正常。双肺清晰，未见明显异常。肋骨完整。'
    }
    print(f"图像路径: {raw_data['image_path']}")
    print(f"医学报告: '{raw_data['report']}'\n")
    
    # 步骤2: 文本预处理
    print("✂️ 步骤2: 文本预处理")
    
    # 简化的清洗函数
    def simple_clean(text):
        # 去除标点，转小写，添加句点
        import re
        text = text.replace('。', ' .').replace('，', '')
        text = re.sub('[.,?;*!%^&_+():-\[\]{}]', '', text)
        return text.lower().strip() + ' .'
    
    cleaned_report = simple_clean(raw_data['report'])
    print(f"清洗后报告: '{cleaned_report}'")
    
    # 分词
    tokens = cleaned_report.split()
    print(f"分词结果: {tokens}")
    
    # 转换为ID（简化的词典）
    vocab = {'心脏': 1, '大小': 2, '正常': 3, '双肺': 4, '清晰': 5, 
             '未': 6, '见': 7, '明显': 8, '异常': 9, '肋骨': 10, 
             '完整': 11, '.': 12, '<unk>': 0}
    
    token_ids = [vocab.get(token, 0) for token in tokens]
    print(f"转换为ID: {token_ids}\n")
    
    # 步骤3: 图像预处理模拟
    print("🖼️ 步骤3: 图像预处理")
    print("假设原始图像大小: 512x512 像素")
    print("预处理步骤:")
    print("  1. 调整大小: 512x512 → 256x256")
    print("  2. 随机裁剪: 256x256 → 224x224")
    print("  3. 随机翻转: 50%概率水平翻转")
    print("  4. 转换张量: HWC格式 → CHW格式")
    print("  5. 标准化: [0,255] → [-2,2]范围\n")
    
    # 步骤4: 批量组装
    print("📦 步骤4: 批量组装")
    print("假设batch_size=16，将16个样本组装成一个批次:")
    print("• 图像批次形状: [16, 3, 224, 224]")
    print("  - 16: 批次大小")
    print("  - 3: RGB三通道") 
    print("  - 224x224: 图像尺寸")
    print("• 文本批次形状: [16, max_length]")
    print("  - 16: 批次大小")
    print("  - max_length: 最长报告的长度（短的会填充）\n")
    
    print("🎯 最终输出:")
    print("模型接收到的数据格式:")
    print("• images: torch.Tensor[16, 3, 224, 224]")
    print("• reports: torch.LongTensor[16, 60]")
    print("• masks: torch.FloatTensor[16, 60]")
    print("\n✅ 数据准备完成，可以开始训练！")

# 运行模拟
data_processing_simulation()

=== 🔄 完整数据处理流程模拟 ===

📁 步骤1: 原始数据
图像路径: datasets/iu_xray/images/CXR1_1_IM-0001/1.png
医学报告: '心脏大小正常。双肺清晰，未见明显异常。肋骨完整。'

✂️ 步骤2: 文本预处理
清洗后报告: '心脏大小正常 双肺清晰未见明显异常 肋骨完整 .'
分词结果: ['心脏大小正常', '双肺清晰未见明显异常', '肋骨完整', '.']
转换为ID: [0, 0, 0, 12]

🖼️ 步骤3: 图像预处理
假设原始图像大小: 512x512 像素
预处理步骤:
  1. 调整大小: 512x512 → 256x256
  2. 随机裁剪: 256x256 → 224x224
  3. 随机翻转: 50%概率水平翻转
  4. 转换张量: HWC格式 → CHW格式
  5. 标准化: [0,255] → [-2,2]范围

📦 步骤4: 批量组装
假设batch_size=16，将16个样本组装成一个批次:
• 图像批次形状: [16, 3, 224, 224]
  - 16: 批次大小
  - 3: RGB三通道
  - 224x224: 图像尺寸
• 文本批次形状: [16, max_length]
  - 16: 批次大小
  - max_length: 最长报告的长度（短的会填充）

🎯 最终输出:
模型接收到的数据格式:
• images: torch.Tensor[16, 3, 224, 224]
• reports: torch.LongTensor[16, 60]
• masks: torch.FloatTensor[16, 60]

✅ 数据准备完成，可以开始训练！


## 本节总结与下节预告 📝

### 🎯 今天我们学到了什么？

1. **Tokenizer分词器**
   - 文本清洗和标准化
   - 词汇表构建和管理
   - 文字与数字的双向转换

2. **DataLoader数据加载器**
   - 批量数据处理
   - 图像预处理流程
   - 高效的数据管道设计

3. **数据增强技术**
   - 提高数据多样性的方法
   - 医学影像的特殊考虑
   - 防止过拟合的策略

### 🧠 关键理解点

- **数据预处理**是深度学习成功的基础
- **标准化**让模型训练更稳定
- **批量处理**大大提高计算效率
- **数据增强**提升模型泛化能力

### 🚀 下节预告

下一节我们将学习R2Gen的核心架构：
1. **视觉特征提取器** - CNN如何"看懂"医学影像
2. **编码器-解码器结构** - 如何从图像生成文本
3. **注意力机制** - 让AI学会"重点关注"

### 💪 练习建议

1. 回顾数据处理的每个步骤，理解其必要性
2. 思考：为什么要将文字转换成数字？
3. 想象一下：如果你是AI，你希望接收什么样格式的数据？

---
**记住：好的数据预处理是模型成功的一半！** 🎯✨

In [ ]:
````xml
<VSCode.Cell language="markdown">
# R2Gen项目学习笔记 - 第二部分：数据处理与文本预处理

## 上节回顾 📚

在第一部分中，我们学习了：
- R2Gen项目的整体架构
- 主函数的执行流程  
- 重要参数的含义
- 深度学习的基础概念

## 本节学习目标 🎯

今天我们将深入学习数据处理的核心模块：

1. **文本分词器 (Tokenizer)** - 如何把医学报告转换成数字
2. **数据加载器 (DataLoader)** - 如何高效管理训练数据
3. **图像预处理** - 如何准备医学影像数据
4. **实际代码演示** - 动手体验数据处理过程

## 为什么数据处理如此重要？ 🤔

就像医生需要先整理病历资料才能诊断一样，AI模型也需要把原始的图像和文字整理成标准格式才能学习！
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 什么是Tokenizer（分词器）？ ✂️

**分词器**是深度学习中处理文本的核心工具，它的作用是：

### 🔄 主要功能
1. **文本清洗** - 去除无用的标点符号和格式
2. **分词切割** - 把句子切分成单词
3. **建立词典** - 为每个单词分配唯一的数字ID
4. **数字转换** - 把文字转换成数字序列

### 🏥 医学报告的特殊性
医学报告有其特殊性：
- 包含专业术语（如"心影增大"、"肺纹理增粗"）
- 格式相对固定（如"心脏：正常大小"）
- 需要标准化处理

### 🎯 为什么需要转换成数字？
计算机只能理解数字，不能直接理解文字。就像：
- "心脏" → 数字 ID: 156
- "正常" → 数字 ID: 89  
- "大小" → 数字 ID: 203
</VSCode.Cell>
<VSCode.Cell language="python">
# 让我们用简单的例子理解Tokenizer的工作原理

def simple_tokenizer_demo():
    """
    演示分词器的基本工作流程
    这是一个简化版本，帮助理解核心概念
    """
    
    print("=== 🔧 分词器工作流程演示 ===\n")
    
    # 步骤1: 原始医学报告
    print("📋 步骤1: 原始医学报告")
    raw_report = "心脏大小正常。肺部清晰，无异常发现。"
    print(f"原始报告: '{raw_report}'\n")
    
    # 步骤2: 文本清洗
    print("🧹 步骤2: 文本清洗")
    # 去除标点符号，转小写，分句
    cleaned = raw_report.replace("。", " . ").replace("，", "").lower()
    print(f"清洗后: '{cleaned}'\n")
    
    # 步骤3: 分词
    print("✂️ 步骤3: 分词切割")
    tokens = cleaned.split()
    print(f"分词结果: {tokens}\n")
    
    # 步骤4: 建立词典（简化版）
    print("📖 步骤4: 建立词典")
    # 实际中会统计所有训练数据中的词汇
    vocab = {
        '<unk>': 0,  # 未知词
        '心脏': 1,
        '大小': 2, 
        '正常': 3,
        '肺部': 4,
        '清晰': 5,
        '无': 6,
        '异常': 7,
        '发现': 8,
        '.': 9
    }
    print("词典示例:")
    for word, idx in vocab.items():
        print(f"  '{word}' → {idx}")
    print()
    
    # 步骤5: 转换为数字序列
    print("🔢 步骤5: 转换为数字序列")
    token_ids = []
    for token in tokens:
        if token in vocab:
            token_ids.append(vocab[token])
        else:
            token_ids.append(vocab['<unk>'])  # 未知词用0表示
    
    print(f"最终数字序列: {token_ids}")
    print("对应关系:")
    for i, (token, token_id) in enumerate(zip(tokens, token_ids)):
        print(f"  '{token}' → {token_id}")
    
    print("\n💡 这样计算机就能'理解'医学报告了！")

# 运行演示
simple_tokenizer_demo()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## R2Gen中的Tokenizer详解 🔍

让我们看看R2Gen项目中真正的Tokenizer是如何工作的：
</VSCode.Cell>
<VSCode.Cell language="python">
# R2Gen Tokenizer 的核心功能分析

def analyze_r2gen_tokenizer():
    """
    分析R2Gen项目中Tokenizer的具体实现
    理解专业的文本处理流程
    """
    
    print("=== 🏥 R2Gen Tokenizer 核心功能 ===\n")
    
    # 功能1: 文本清洗
    print("🧼 功能1: 专业的文本清洗")
    print("• 去除多余的标点符号和数字编号")
    print("• 处理换行符和多余空格")
    print("• 统一转换为小写")
    print("• 特殊处理医学报告的格式")
    
    # 示例清洗过程
    original_text = "1. Heart size is normal.. 2. Lungs are clear."
    print(f"\n原始文本: '{original_text}'")
    
    # 模拟清洗步骤
    step1 = original_text.replace('..', '.').replace('1. ', '').replace('. 2. ', '. ')
    step2 = step1.strip().lower()
    # 去除标点符号（简化版）
    import re
    step3 = re.sub('[.,?;*!%^&_+():-\[\]{}]', '', step2.replace('"', ''))
    final_clean = step3 + ' .'
    
    print(f"清洗后: '{final_clean}'\n")
    
    # 功能2: 词汇表构建
    print("📚 功能2: 智能词汇表构建")
    print("• 统计所有训练数据中的词汇出现频率")
    print("• 只保留出现次数 >= threshold 的词汇")
    print("• 自动过滤掉罕见词汇，提高训练效率")
    print("• 添加特殊标记：<unk>（未知词）\n")
    
    # 功能3: 编码解码
    print("🔄 功能3: 双向编码解码")
    print("• encode: 文字 → 数字序列")
    print("• decode: 数字序列 → 文字")
    print("• 支持批量处理多个报告")
    print("• 自动添加起始和结束标记\n")
    
    # 功能4: 特殊处理
    print("⚡ 功能4: 智能特殊处理")
    print("• 处理未知词汇：遇到词典中没有的词用<unk>代替")
    print("• 长度控制：超长报告会被截断")
    print("• 填充处理：短报告会被填充到固定长度")
    
    print("\n🎯 核心优势：")
    print("• 专门针对医学文本优化")
    print("• 处理速度快，内存占用少")
    print("• 可以处理IU X-Ray和MIMIC-CXR两种不同数据集")

# 运行分析
analyze_r2gen_tokenizer()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 数据加载器 (DataLoader) 解析 📦

**数据加载器**是训练过程中的"后勤部门"，负责：
- 🔄 批量加载数据
- 🖼️ 图像预处理  
- 📝 文本预处理
- 🔀 数据打乱和采样
</VSCode.Cell>
<VSCode.Cell language="python">
# DataLoader 工作原理详解

def explain_dataloader():
    """
    解释数据加载器的工作原理和重要性
    用生活化的比喻帮助理解
    """
    
    print("=== 🏭 DataLoader: 训练数据的'生产流水线' ===\n")
    
    print("🎯 核心作用:")
    print("就像工厂的流水线一样，DataLoader负责:")
    print("• 从仓库（数据集）中取出原材料（原始数据）")
    print("• 按照标准流程加工（预处理）")
    print("• 打包成标准规格（批量处理）")
    print("• 源源不断供应给生产线（模型训练）\n")
    
    print("📊 批量处理 (Batch Processing):")
    print("• 不是一张一张图片处理，而是一次处理16张（batch_size=16）")
    print("• 就像医生不是一个一个看病人，而是安排一批病人同时检查")
    print("• 大大提高了处理效率\n")
    
    print("🖼️ 图像预处理流程:")
    print("训练时:")
    print("  1. Resize(256) - 调整图片大小到256x256")
    print("  2. RandomCrop(224) - 随机裁剪到224x224（数据增强）")
    print("  3. RandomHorizontalFlip() - 随机水平翻转（数据增强）") 
    print("  4. ToTensor() - 转换为张量格式")
    print("  5. Normalize() - 标准化像素值")
    print()
    print("测试时:")
    print("  1. Resize(224, 224) - 直接调整到224x224（不随机）")
    print("  2. ToTensor() - 转换为张量")
    print("  3. Normalize() - 标准化\n")
    
    print("🧮 为什么要标准化？")
    print("• 原始像素值范围: 0-255")  
    print("• 标准化后范围: 大约-2到2之间")
    print("• 这样神经网络训练更稳定，收敛更快")
    print("• 就像把不同单位的数据（米、厘米、毫米）统一成一个标准\n")
    
    print("🔀 数据打乱 (Shuffle):")
    print("• 训练时: shuffle=True（打乱数据顺序）")
    print("• 测试时: shuffle=False（保持固定顺序）")
    print("• 为什么要打乱？避免模型记住数据顺序而不是真正学习规律")

# 运行解释
explain_dataloader()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 实际数据处理演示 🛠️

让我们动手体验一下数据处理的完整流程：
</VSCode.Cell>
<VSCode.Cell language="python">
# 模拟完整的数据处理流程

def data_processing_simulation():
    """
    模拟R2Gen中的完整数据处理流程
    让大家体验从原始数据到训练数据的转换过程
    """
    
    print("=== 🔄 完整数据处理流程模拟 ===\n")
    
    # 模拟原始数据
    print("📁 步骤1: 原始数据")
    raw_data = {
        'image_path': 'datasets/iu_xray/images/CXR1_1_IM-0001/1.png',
        'report': '心脏大小正常。双肺清晰，未见明显异常。肋骨完整。'
    }
    print(f"图像路径: {raw_data['image_path']}")
    print(f"医学报告: '{raw_data['report']}'\n")
    
    # 步骤2: 文本预处理
    print("✂️ 步骤2: 文本预处理")
    
    # 简化的清洗函数
    def simple_clean(text):
        # 去除标点，转小写，添加句点
        import re
        text = text.replace('。', ' .').replace('，', '')
        text = re.sub('[.,?;*!%^&_+():-\[\]{}]', '', text)
        return text.lower().strip() + ' .'
    
    cleaned_report = simple_clean(raw_data['report'])
    print(f"清洗后报告: '{cleaned_report}'")
    
    # 分词
    tokens = cleaned_report.split()
    print(f"分词结果: {tokens}")
    
    # 转换为ID（简化的词典）
    vocab = {'心脏': 1, '大小': 2, '正常': 3, '双肺': 4, '清晰': 5, 
             '未': 6, '见': 7, '明显': 8, '异常': 9, '肋骨': 10, 
             '完整': 11, '.': 12, '<unk>': 0}
    
    token_ids = [vocab.get(token, 0) for token in tokens]
    print(f"转换为ID: {token_ids}\n")
    
    # 步骤3: 图像预处理模拟
    print("🖼️ 步骤3: 图像预处理")
    print("假设原始图像大小: 512x512 像素")
    print("预处理步骤:")
    print("  1. 调整大小: 512x512 → 256x256")
    print("  2. 随机裁剪: 256x256 → 224x224")
    print("  3. 随机翻转: 50%概率水平翻转")
    print("  4. 转换张量: HWC格式 → CHW格式")
    print("  5. 标准化: [0,255] → [-2,2]范围\n")
    
    # 步骤4: 批量组装
    print("📦 步骤4: 批量组装")
    print("假设batch_size=16，将16个样本组装成一个批次:")
    print("• 图像批次形状: [16, 3, 224, 224]")
    print("  - 16: 批次大小")
    print("  - 3: RGB三通道") 
    print("  - 224x224: 图像尺寸")
    print("• 文本批次形状: [16, max_length]")
    print("  - 16: 批次大小")
    print("  - max_length: 最长报告的长度（短的会填充）\n")
    
    print("🎯 最终输出:")
    print("模型接收到的数据格式:")
    print("• images: torch.Tensor[16, 3, 224, 224]")
    print("• reports: torch.LongTensor[16, 60]")
    print("• masks: torch.FloatTensor[16, 60]")
    print("\n✅ 数据准备完成，可以开始训练！")

# 运行模拟
data_processing_simulation()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 数据增强技术 🔄

**数据增强**是提高模型性能的重要技术：
</VSCode.Cell>
<VSCode.Cell language="python">
# 数据增强技术详解

def explain_data_augmentation():
    """
    解释数据增强技术及其在医学影像中的应用
    """
    
    print("=== 🔄 数据增强: 让AI见多识广 ===\n")
    
    print("🤔 什么是数据增强？")
    print("通过对原始数据进行微小的变换，创造更多训练样本")
    print("就像让医学生看同一个X光片的不同角度和光照条件\n")
    
    print("🏥 医学影像中的数据增强技术:")
    print()
    
    print("1. 🔧 RandomCrop (随机裁剪)")
    print("   • 从256x256的图像中随机裁剪224x224区域")
    print("   • 让模型学会关注图像的不同部分")
    print("   • 提高模型的泛化能力")
    print()
    
    print("2. 🔄 RandomHorizontalFlip (随机水平翻转)")
    print("   • 50%的概率将图像水平翻转")
    print("   • 对于胸部X光片，左右翻转通常不影响诊断")
    print("   • 有效增加数据的多样性")
    print()
    
    print("3. 📏 Resize (尺寸调整)")
    print("   • 将不同尺寸的医学影像统一到固定大小")
    print("   • 确保模型输入的一致性")
    print("   • 训练时: 256x256，测试时: 224x224")
    print()
    
    print("4. 🎨 Normalize (标准化)")
    print("   • 使用ImageNet的均值和标准差")
    print("   • mean=[0.485, 0.456, 0.406]")
    print("   • std=[0.229, 0.224, 0.225]")
    print("   • 这样可以利用在ImageNet上预训练的模型")
    print()
    
    print("⚠️ 医学影像数据增强的注意事项:")
    print("• 不能随意旋转（可能改变病理特征）")
    print("• 不能改变像素值分布（可能影响诊断）")
    print("• 保持医学影像的真实性和可解释性")
    print()
    
    print("💡 为什么数据增强有效？")
    print("• 增加训练数据的多样性")
    print("• 提高模型的泛化能力") 
    print("• 减少过拟合现象")
    print("• 让模型更鲁棒（robust）")

# 运行解释
explain_data_augmentation()
</VSCode.Cell>
<VSCode.Cell language="markdown">
## 本节总结与下节预告 📝

### 🎯 今天我们学到了什么？

1. **Tokenizer分词器**
   - 文本清洗和标准化
   - 词汇表构建和管理
   - 文字与数字的双向转换

2. **DataLoader数据加载器**
   - 批量数据处理
   - 图像预处理流程
   - 高效的数据管道设计

3. **数据增强技术**
   - 提高数据多样性的方法
   - 医学影像的特殊考虑
   - 防止过拟合的策略

### 🧠 关键理解点

- **数据预处理**是深度学习成功的基础
- **标准化**让模型训练更稳定
- **批量处理**大大提高计算效率
- **数据增强**提升模型泛化能力

### 🚀 下节预告

下一节我们将学习R2Gen的核心架构：
1. **视觉特征提取器** - CNN如何"看懂"医学影像
2. **编码器-解码器结构** - 如何从图像生成文本
3. **注意力机制** - 让AI学会"重点关注"

### 💪 练习建议

1. 回顾数据处理的每个步骤，理解其必要性
2. 思考：为什么要将文字转换成数字？
3. 想象一下：如果你是AI，你希望接收什么样格式的数据？

---
**记住：好的数据预处理是模型成功的一半！** 🎯✨
</VSCode.Cell>
````